In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LinearRegression
from sklearn.metrics import f1_score
from sklearn.tree import export_text
import mglearn
from dashboard_one import *
from dash_model_two import *
from dashboard_three import *
from feature_selection import *
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate

###  brain_volume_ratio_to_baseline_____VS_____diagnosischanges from every visit


In [2]:
com_col = ['Phase', 'RID', 'VISCODE2','PTID']
target = 'DIAG_GROUP'

In [3]:
df = pd.read_csv('main_file_1206_wz.csv').iloc[:,1:].drop(['NPIKSEV'],axis=1)
df = df[df['DIAG_GROUP'].notna()].reset_index().drop(['index'],axis=1)   # keep the rows where DXCHANGE is not nan
df

,RID,DIAG,VISCODE2,PTID,AGE,PTGENDER,PTEDUCAT,Phase,EXAMDATE,Ventricles,...,NPIK7,NPIK8,NPIK9A,NPIK9B,NPIK9C,NPIKTOT,OSA,PREV_DIAG,DIAG_CHANGED,DIAG_GROUP
0,2,NL,sc,011_S_0002,74.3,Male,16.0,ADNI1,2005-08-09,118233.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NL,False,NL-NL
1,2,NL,bl,011_S_0002,74.3,Male,16.0,ADNI1,2005-08-09,118233.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NL,False,NL-NL
2,2,NL,m06,011_S_0002,74.3,Male,16.0,ADNI1,2006-06-03,118233.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NL,False,NL-NL
3,2,NL,m36,011_S_0002,74.3,Male,16.0,ADNI1,2008-08-27,118233.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NL,False,NL-NL
4,2,NL,m60,011_S_0002,74.3,Male,16.0,ADNI1,2010-09-22,118233.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NL,False,NL-NL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22196,7102,NL,sc,341_S_7102,NaN,NaN,NaN,ADNI3,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NL,False,NL-NL
22197,7103,NL,sc,013_S_7103,NaN,NaN,NaN,ADNI3,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NL,False,NL-NL
22198,7104,NL,sc,153_S_7104,NaN,NaN,NaN,ADNI3,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NL,False,NL-NL
22199,7105,MCI,sc,035_S_7105,NaN,NaN,NaN,ADNI3,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,MCI,False,MCI-MCI


### brain volume______VS______DXCHANGE

In [4]:
df.columns

Index(['RID', 'DIAG', 'VISCODE2', 'PTID', 'AGE', 'PTGENDER', 'PTEDUCAT',
       'Phase', 'EXAMDATE', 'Ventricles', 'Hippocampus', 'WholeBrain',
       'Entorhinal', 'Fusiform', 'MidTemp', 'ICV', 'Ventricles_bl',
       'Hippocampus_bl', 'WholeBrain_bl', 'Entorhinal_bl', 'Fusiform_bl',
       'MidTemp_bl', 'ICV_bl', 'ABETA', 'TAU', 'PTAU', 'ABETA_bl', 'TAU_bl',
       'PTAU_bl', 'ratio_Ventricles_bl', 'ratio_Hippocampus_bl',
       'ratio_WholeBrain_bl', 'ratio_Entorhinal_bl', 'ratio_Fusiform_bl',
       'ratio_ICV_bl', 'ratio_ABETA_bl', 'ratio_TAU_bl', 'ratio_PTAU_bl',
       'EXAMDATE_bl', 'duration_in_days', 'Ventricles_reduction_per_year',
       'Hippocampus_reduction_per_year', 'wholebrain_reduction_per_year',
       'Entorhinal_reduction_per_year', 'Fusiform_reduction_per_year',
       'ICV_reduction_per_year', 'ABETA_reduction_per_year',
       'TAU_reduction_per_year', 'PTAU_reduction_per_year', 'insomnia',
       'NPIK1', 'NPIK2', 'NPIK3', 'NPIK4', 'NPIK5', 'NPIK6', 'NPIK7', '

In [5]:
col_lst = [ target,'Ventricles_reduction_per_year',
       'Hippocampus_reduction_per_year', 'wholebrain_reduction_per_year',
       'Entorhinal_reduction_per_year', 'Fusiform_reduction_per_year',
       'ICV_reduction_per_year']
df1 = df[com_col + col_lst].set_index(com_col).dropna(how='any',axis=0).reset_index()
df1

,Phase,RID,VISCODE2,PTID,DIAG_GROUP,Ventricles_reduction_per_year,Hippocampus_reduction_per_year,wholebrain_reduction_per_year,Entorhinal_reduction_per_year,Fusiform_reduction_per_year,ICV_reduction_per_year
0,ADNI1,3,sc,011_S_0003,AD-AD,-0.182722,-0.092713,0.102313,-1.378880,0.276962,0.028829
1,ADNI1,3,bl,011_S_0003,AD-AD,-0.182722,-0.092713,0.102313,-1.378880,0.276962,0.028829
2,ADNI1,3,m06,011_S_0003,AD-AD,-0.182722,-0.092713,0.102313,-1.378880,0.276962,0.028829
3,ADNI1,3,m12,011_S_0003,AD-AD,-0.065013,0.030457,0.030261,0.108878,0.057333,0.008783
4,ADNI1,3,m18,011_S_0003,AD-AD,-0.065013,0.030457,0.030261,0.108878,0.057333,0.008783
...,...,...,...,...,...,...,...,...,...,...,...
15400,ADNI3,6896,bl,021_S_6896,NL-NL,0.009125,-0.009420,-0.008385,0.019959,-0.037201,-0.024754
15401,ADNI3,6896,m12,021_S_6896,NL-NL,0.009125,-0.009420,-0.008385,0.019959,-0.037201,-0.024754
15402,ADNI3,6897,sc,036_S_6897,MCI-MCI,-0.020030,-0.002037,0.006370,-0.137194,0.015551,0.000041
15403,ADNI3,6897,bl,036_S_6897,MCI-MCI,-0.020030,-0.002037,0.006370,-0.137194,0.015551,0.000041


In [6]:
col_lst_2 = [ target,'ratio_Ventricles_bl', 'ratio_Hippocampus_bl',
       'ratio_WholeBrain_bl', 'ratio_Entorhinal_bl', 'ratio_Fusiform_bl',
       'ratio_ICV_bl']
df2 = df[com_col + col_lst_2].set_index(com_col).dropna(how='any',axis=0).reset_index()
df2

,Phase,RID,VISCODE2,PTID,DIAG_GROUP,ratio_Ventricles_bl,ratio_Hippocampus_bl,ratio_WholeBrain_bl,ratio_Entorhinal_bl,ratio_Fusiform_bl,ratio_ICV_bl
0,ADNI1,2,sc,011_S_0002,NL-NL,1.0,1.0,1.0,1.0,1.0,1.0
1,ADNI1,2,bl,011_S_0002,NL-NL,1.0,1.0,1.0,1.0,1.0,1.0
2,ADNI1,2,m06,011_S_0002,NL-NL,1.0,1.0,1.0,1.0,1.0,1.0
3,ADNI1,2,m36,011_S_0002,NL-NL,1.0,1.0,1.0,1.0,1.0,1.0
4,ADNI1,2,m60,011_S_0002,NL-NL,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
16931,ADNI3,7033,bl,067_S_7033,AD-AD,1.0,1.0,1.0,1.0,1.0,1.0
16932,ADNI3,7048,sc,011_S_7048,NL-NL,1.0,1.0,1.0,1.0,1.0,1.0
16933,ADNI3,7048,bl,011_S_7048,NL-NL,1.0,1.0,1.0,1.0,1.0,1.0
16934,ADNI3,7066,sc,033_S_7066,AD-AD,1.0,1.0,1.0,1.0,1.0,1.0


In [7]:
col_lst_3 = [ target,'ratio_Ventricles_bl', 'ratio_Hippocampus_bl',
       'ratio_WholeBrain_bl', 'ratio_Entorhinal_bl', 'ratio_Fusiform_bl',
       'ratio_ICV_bl','Ventricles_reduction_per_year',
       'Hippocampus_reduction_per_year', 'wholebrain_reduction_per_year',
       'Entorhinal_reduction_per_year', 'Fusiform_reduction_per_year',
       'ICV_reduction_per_year']
df3 = df[com_col + col_lst_3].set_index(com_col).dropna(how='any',axis=0).reset_index()
df3

,Phase,RID,VISCODE2,PTID,DIAG_GROUP,ratio_Ventricles_bl,ratio_Hippocampus_bl,ratio_WholeBrain_bl,ratio_Entorhinal_bl,ratio_Fusiform_bl,ratio_ICV_bl,Ventricles_reduction_per_year,Hippocampus_reduction_per_year,wholebrain_reduction_per_year,Entorhinal_reduction_per_year,Fusiform_reduction_per_year,ICV_reduction_per_year
0,ADNI1,3,sc,011_S_0003,AD-AD,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,-0.182722,-0.092713,0.102313,-1.378880,0.276962,0.028829
1,ADNI1,3,bl,011_S_0003,AD-AD,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,-0.182722,-0.092713,0.102313,-1.378880,0.276962,0.028829
2,ADNI1,3,m06,011_S_0003,AD-AD,1.047057,1.023877,0.973651,1.355109,0.928673,0.992576,-0.182722,-0.092713,0.102313,-1.378880,0.276962,0.028829
3,ADNI1,3,m12,011_S_0003,AD-AD,1.065013,0.969543,0.969739,0.891122,0.942667,0.991217,-0.065013,0.030457,0.030261,0.108878,0.057333,0.008783
4,ADNI1,3,m18,011_S_0003,AD-AD,1.065013,0.969543,0.969739,0.891122,0.942667,0.991217,-0.065013,0.030457,0.030261,0.108878,0.057333,0.008783
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15400,ADNI3,6896,bl,021_S_6896,NL-NL,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.009125,-0.009420,-0.008385,0.019959,-0.037201,-0.024754
15401,ADNI3,6896,m12,021_S_6896,NL-NL,0.988225,1.012156,1.010820,0.974245,1.048005,1.031943,0.009125,-0.009420,-0.008385,0.019959,-0.037201,-0.024754
15402,ADNI3,6897,sc,036_S_6897,MCI-MCI,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,-0.020030,-0.002037,0.006370,-0.137194,0.015551,0.000041
15403,ADNI3,6897,bl,036_S_6897,MCI-MCI,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,-0.020030,-0.002037,0.006370,-0.137194,0.015551,0.000041


In [8]:
np.sum(df1.isna())   # check nan

Phase                             0
RID                               0
VISCODE2                          0
PTID                              0
DIAG_GROUP                        0
Ventricles_reduction_per_year     0
Hippocampus_reduction_per_year    0
wholebrain_reduction_per_year     0
Entorhinal_reduction_per_year     0
Fusiform_reduction_per_year       0
ICV_reduction_per_year            0
dtype: int64

In [9]:
df1.groupby(target).count()

,Phase,RID,VISCODE2,PTID,Ventricles_reduction_per_year,Hippocampus_reduction_per_year,wholebrain_reduction_per_year,Entorhinal_reduction_per_year,Fusiform_reduction_per_year,ICV_reduction_per_year
DIAG_GROUP,,,,,,,,,,
AD-AD,2406,2406,2406,2406,2406,2406,2406,2406,2406,2406
AD-MCI,28,28,28,28,28,28,28,28,28,28
AD-NL,2,2,2,2,2,2,2,2,2,2
MCI-AD,215,215,215,215,215,215,215,215,215,215
MCI-MCI,6382,6382,6382,6382,6382,6382,6382,6382,6382,6382
MCI-NL,72,72,72,72,72,72,72,72,72,72
NL-MCI,86,86,86,86,86,86,86,86,86,86
NL-NL,6214,6214,6214,6214,6214,6214,6214,6214,6214,6214


##### 5 models using only reduction_per_year features;  the features could be explained by 2 components
#####

In [10]:
g1 = df1.loc[df1[target].isin(['NL-MCI','NL-NL'])].reset_index(drop=True)
g2 = df1.loc[df1[target].isin(['MCI-AD','MCI-MCI','MCI-NL'])].reset_index(drop=True)
g3 = df1.loc[df1[target].isin(['MCI-AD','MCI-MCI'])].reset_index(drop=True)
g4 = df1.loc[df1[target].isin(['MCI-AD','MCI-NL'])].reset_index(drop=True)
g5 = df1.loc[df1[target].isin(['MCI-MCI','MCI-NL'])].reset_index(drop=True)

g1.shape,g2.shape,g3.shape,g4.shape, g5.shape   

((6300, 11), (6669, 11), (6597, 11), (287, 11), (6454, 11))

##### 5 models using only ratio to baseline features ; the features could be explained by 2 components
#####

In [11]:
g21 = df2.loc[df2[target].isin(['NL-MCI','NL-NL'])].reset_index(drop=True)
g22 = df2.loc[df2[target].isin(['MCI-AD','MCI-MCI','MCI-NL'])].reset_index(drop=True)
g23 = df2.loc[df2[target].isin(['MCI-AD','MCI-MCI'])].reset_index(drop=True)
g24 = df2.loc[df2[target].isin(['MCI-AD','MCI-NL'])].reset_index(drop=True)
g25 = df2.loc[df2[target].isin(['MCI-MCI','MCI-NL'])].reset_index(drop=True)

g21.shape,g22.shape,g23.shape,g24.shape, g25.shape   

((7029, 11), (7224, 11), (7150, 11), (289, 11), (7009, 11))

##### 5 models using baseline ratio and reduction_per_year features ; the features could be explained by 2 components
#####

In [12]:
g31 = df3.loc[df3[target].isin(['NL-MCI','NL-NL'])].reset_index(drop=True)
g32 = df3.loc[df3[target].isin(['MCI-AD','MCI-MCI','MCI-NL'])].reset_index(drop=True)
g33 = df3.loc[df3[target].isin(['MCI-AD','MCI-MCI'])].reset_index(drop=True)
g34 = df3.loc[df3[target].isin(['MCI-AD','MCI-NL'])].reset_index(drop=True)
g35 = df3.loc[df3[target].isin(['MCI-MCI','MCI-NL'])].reset_index(drop=True)

g31.shape,g32.shape,g33.shape,g34.shape, g35.shape 

((6300, 17), (6669, 17), (6597, 17), (287, 17), (6454, 17))

### oversampling and undersampling

- functions
    - models(df,drop_lst,target) : under sampling, split, scale, pca, models
    - cv_models(df,drop_lst,target,k): under sampling, NOT SPLIT, scale, pca, models with cross-validation 

In [23]:
drop_lst = ['Phase', 'RID', 'VISCODE2', 'PTID',target]
k = 10

In [24]:
drop_lst[0:-1]

['Phase', 'RID', 'VISCODE2', 'PTID']

In [14]:
cv_models(g1,drop_lst,target,k)

# pca  C = 0.1
          #- lbfgs_L2, average weighted f1-score of 10-cross validation:0.621

After undersampling data size is 172 ; Resampled dataset shape Counter({'NL-MCI': 86, 'NL-NL': 86})

2 principle components are needed to explain 90% of the data

- Using original dataset:
    - Logistic regression
       - C = 0.001
          - lbfgs_L2, average weighted f1-score of 10-cross validation:0.351
          - saga_L1, average weighted f1-score of 10-cross validation:0.334
          - newton-cg_L2, average weighted f1-score of 10-cross validation:0.340
       - C = 0.01
          - lbfgs_L2, average weighted f1-score of 10-cross validation:0.415
          - saga_L1, average weighted f1-score of 10-cross validation:0.334
          - newton-cg_L2, average weighted f1-score of 10-cross validation:0.415
       - C = 0.1
          - lbfgs_L2, average weighted f1-score of 10-cross validation:0.525
          - saga_L1, average weighted f1-score of 10-cross validation:0.327
          - newton-cg_L2, average weighted f1-score of 10-cross validation:0.525
       - C = 1
          - lb

KeyboardInterrupt: 

In [ ]:
cv_models(g1,drop_lst,target,k)

In [ ]:
cv_models(g1,drop_lst,target,k)
# pca C = 0.1
#          - lbfgs_L2, average weighted f1-score of 10-cross validation:0.621

In [ ]:
cv_models(g2,drop_lst,target,k)  # no good score

In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LinearRegression
from sklearn.metrics import f1_score
from sklearn.tree import export_text
import mglearn
from dashboard_one import *
from feature_selection import *
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate


# In[5]:


## models_os(df,drop_lst,target)

def oversampling_split_scale_data(df,drop_lst,target):
    '''
    oversampling data, split data (4:1),data scaling, pca components which could explains 90% data
    ----------------------------------
    df: the full dataframe
    target: the target feature name
    -----------
    Outputs: X_train,X_test,X_train_scaled,X_test_scaled,X_train_pca,X_test_pca,y_train,y_test
       '''    
    # split data
    drop_lst_2 = drop_lst[0:-1]
    train, test = train_test_split(df,test_size=0.2)  
    X_train = train.drop(drop_lst,axis=1)
    y_train = train[target]
    X_test = test.drop(drop_lst,axis=1)
    y_test = test[target]

    # oversampling
    ros = RandomOverSampler(random_state=432)
    X_oversampled,y_oversampled = ros.fit_resample(X_train,y_train)
    print('After oversampling, train data size is',len(X_oversampled),'; Resampled dataset shape %s' % Counter(y_oversampled))
    
    # data scaling
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train_scaled = scaler.transform(X_oversampled)
    X_test_scaled = scaler.transform(X_test)
    # pca components
    pca = PCA(n_components=X_train_scaled.shape[1]) # keep all n principal components 
    pca.fit(X_train_scaled) # fit PCA model with scaled data
    X_pca = pca.transform(X_train_scaled)  #transform data onto the first two principal components
    ex_ratio = pca.explained_variance_ratio_
    cum_sum = 0
    for i in range(len(ex_ratio)):
        cum_sum += ex_ratio[i]
        if cum_sum >= 0.9:  # if it could explain 90% of the data, then stop
            break
    n_com = i         
         # PCA with first n_com components
    pca = PCA(n_com)
    pca.fit(X_train_scaled)
    X_train_pca = pca.transform(X_train_scaled)
    X_test_pca = pca.transform(X_test_scaled)
    '''   #plot 
    plt.bar(range(1,len(pca.explained_variance_ratio_ )+1),pca.explained_variance_ratio_ )
    plt.ylabel('Explained variance ratio')
    plt.xlabel('Components')
    plt.plot(range(1,len(pca.explained_variance_ )+1),
             np.cumsum(pca.explained_variance_ratio_),
             c='red',
             label="Cumulative Explained Variance ratio")
    plt.legend(loc='upper left')'''
    print('\n{} principle components are needed to explain 90% of the data\n'.format(n_com))  
    print('Output dataframes sequence: X_train,X_test,X_train_scaled,X_test_scaled,X_train_pca,X_test_pca,y_train,y_test')
    X_labels = ['original dataset','scaled dataset','%s pca-components'%n_com]
    return X_oversampled,X_test,X_train_scaled,X_test_scaled,X_train_pca,X_test_pca,y_oversampled,y_test,X_labels  

def models_os(df,drop_lst,target):
    '''
    for splitted data
    '''
    res = oversampling_split_scale_data(df,drop_lst,target)
    y_train = res[6]
    y_test = res[7]
    X_labels = res[8]
    for i in range(len(X_labels)):
        print('- Using {}:'.format(X_labels[i]))
        X_train = res[2*i]
        X_test = res[2*i+1]
        # logistic regression
        C_lst = [0.001,0.01,0.1,1,10,100,1000]
        print('    - Logistic regression')
        for i in range(len(C_lst)):
            print('       - C = {}'.format(C_lst[i]))
            logreg = LogisticRegression(C=C_lst[i],solver='lbfgs',multi_class='auto',penalty='l2',max_iter=10000).fit(X_train,y_train)
            print('          - lbfgs_L2, Training set f1-score:{:.3f}, Test set f1-score: {:.3f}'
                  .format(f1_score(logreg.predict(X_train),y_train,average='weighted'),f1_score(logreg.predict(X_test),y_test,average='weighted')))
            logreg = LogisticRegression(C=C_lst[i],solver='saga',multi_class='auto',penalty='l1',max_iter=10000).fit(X_train,y_train)
            print('          - saga_L1, Training set f1-score:{:.3f}, Test set f1-score: {:.3f}'
                  .format(f1_score(logreg.predict(X_train),y_train,average='weighted'),f1_score(logreg.predict(X_test),y_test,average='weighted')))
            logreg = LogisticRegression(C=C_lst[i],solver='newton-cg',multi_class='auto',penalty='l2',max_iter=10000).fit(X_train,y_train)
            print('          - newton-cg_L2, Training set f1-score:{:.3f}, Test set f1-score: {:.3f}'
                  .format(f1_score(logreg.predict(X_train),y_train,average='weighted'),f1_score(logreg.predict(X_test),y_test,average='weighted')))

        # decision tree
        print('    - Decision tree')
        for i in range(1,15):
            dtree = DecisionTreeClassifier(random_state=0,max_depth=i,criterion='gini')
            dtree.fit(X_train,y_train)
            print('          - tree depth: {:.3f}. f1-score on training data: {:.3f} f1-score on test data: {:.3f}'
              .format(i,f1_score(dtree.predict(X_train),y_train,average='weighted'),f1_score(dtree.predict(X_test),y_test,average='weighted')))
        # random forest
        print('    - Random forest')
        for i in range(1,20):   
            m= 5*i
            forest = RandomForestClassifier(n_estimators=m,random_state=5862)
            forest.fit(X_train,y_train)
            print('          - {}trees. f1-score on training data: {:.3f} f1-score on test data: {:.3f}'
              .format(m,f1_score(forest.predict(X_train),y_train,average='weighted'),f1_score(forest.predict(X_test),y_test,average='weighted')))
        # MLP  
        print('    - MLP')
        hls = [[50,50],[20,20]] # hidden layer size  ,[100,100],[50,50,50]
        for i in range(len(hls)):
            mlp = MLPClassifier(solver='lbfgs',random_state=460,hidden_layer_sizes = hls[i],max_iter=20000)
            mlp.fit(X_train,y_train)
            print('          - hidden layer size{}. f1-score on training data: {:.3f} f1-score on test data: {:.3f}'.format(hls[i],f1_score(mlp.predict(X_train),y_train,average='weighted'),f1_score(mlp.predict(X_test),y_test,average='weighted'))) 
            

models_os(g2,drop_lst,target)

After oversampling, train data size is 15312 ; Resampled dataset shape Counter({'MCI-MCI': 5104, 'MCI-NL': 5104, 'MCI-AD': 5104})

3 principle components are needed to explain 90% of the data

Output dataframes sequence: X_train,X_test,X_train_scaled,X_test_scaled,X_train_pca,X_test_pca,y_train,y_test
- Using original dataset:
    - Logistic regression
       - C = 0.001
          - lbfgs_L2, Training set f1-score:0.471, Test set f1-score: 0.243
          - saga_L1, Training set f1-score:0.500, Test set f1-score: 0.061
          - newton-cg_L2, Training set f1-score:0.471, Test set f1-score: 0.245
       - C = 0.01
          - lbfgs_L2, Training set f1-score:0.495, Test set f1-score: 0.371
          - saga_L1, Training set f1-score:0.442, Test set f1-score: 0.187
          - newton-cg_L2, Training set f1-score:0.495, Test set f1-score: 0.371
       - C = 0.1
          - lbfgs_L2, Training set f1-score:0.467, Test set f1-score: 0.352
          - saga_L1, Training set f1-score:0.464, Tes

          - tree depth: 14.000. f1-score on training data: 0.888 f1-score on test data: 0.547
    - Random forest
          - 5trees. f1-score on training data: 1.000 f1-score on test data: 0.967
          - 10trees. f1-score on training data: 1.000 f1-score on test data: 0.973
          - 15trees. f1-score on training data: 1.000 f1-score on test data: 0.975
          - 20trees. f1-score on training data: 1.000 f1-score on test data: 0.975
          - 25trees. f1-score on training data: 1.000 f1-score on test data: 0.974
          - 30trees. f1-score on training data: 1.000 f1-score on test data: 0.973
          - 35trees. f1-score on training data: 1.000 f1-score on test data: 0.973
          - 40trees. f1-score on training data: 1.000 f1-score on test data: 0.974
          - 45trees. f1-score on training data: 1.000 f1-score on test data: 0.974
          - 50trees. f1-score on training data: 1.000 f1-score on test data: 0.974
          - 55trees. f1-score on training data: 1.000 f1-

In [ ]:
res = usampling_scale_data(g3,drop_lst,target)     
X = res[0]  # unscaled input
y = res[3]
clf = RandomForestClassifier(n_estimators =85, random_state = 5862)
title_label = '10-fold crossvalidation random forest (85 trees)'
feature_importance(X,y,clf,10,title_label)
plt.suptitle('Target: MCI-AD; MCI-MCI')

In [ ]:
cv_models(g3,drop_lst,target,k)

In [ ]:
# - 85trees. average weighted f1-score of 10-cross validation:0.718

res = usampling_scale_data(g3,drop_lst,target)     
X = res[0]  # unscaled input
y = res[3]
clf = RandomForestClassifier(n_estimators =85, random_state = 5862)
title_label = '10-fold crossvalidation random forest (85 trees)'
feature_importance(X,y,clf,10,title_label)
plt.suptitle('Target: MCI-AD; MCI-MCI')

In [ ]:
cv_models(g4,drop_lst,target,k)

In [ ]:
# - 90trees. average weighted f1-score of 10-cross validation:0.744

res = usampling_scale_data(g4,drop_lst,target)     
X = res[0]  # unscaled input
y = res[3]
clf = RandomForestClassifier(n_estimators =90, random_state = 5862)
title_label = '10-fold crossvalidation random forest (90 trees)'
feature_importance(X,y,clf,10,title_label)
plt.suptitle('Target: MCI-AD; MCI-NL')

In [ ]:
cv_models(g5,drop_lst,target,k)   # no good score

In [ ]:
cv_models(g21,drop_lst,target,k)

# - 10trees. average weighted f1-score of 10-cross validation:0.609

In [ ]:
cv_models(g22,drop_lst,target,k)  # no good score

In [ ]:
cv_models(g23,drop_lst,target,k)

In [ ]:
#  - 90trees. average weighted f1-score of 10-cross validation:0.780
res = usampling_scale_data(g23,drop_lst,target)     
X = res[0]  # unscaled input
y = res[3]
clf = RandomForestClassifier(n_estimators =90, random_state = 5862)
title_label = '10-fold crossvalidation random forest (90 trees)'
feature_importance(X,y,clf,10,title_label)
plt.suptitle('Target: MCI-AD: 215, MCI-MCI: 215')

In [ ]:
cv_models(g24,drop_lst,target,k)

In [ ]:
# - 45trees. average weighted f1-score of 10-cross validation:0.699
res = usampling_scale_data(g24,drop_lst,target)     
X = res[0]  # unscaled input
y = res[3]
clf = RandomForestClassifier(n_estimators =45, random_state = 5862)
title_label = '10-fold crossvalidation random forest (45 trees)'
feature_importance(X,y,clf,10,title_label)
plt.suptitle('Target: MCI-AD: 74, MCI-NL: 74')

In [ ]:
cv_models(g25,drop_lst,target,k)  

In [ ]:
# - 80trees. average weighted f1-score of 10-cross validation:0.755
res = usampling_scale_data(g25,drop_lst,target)     
X = res[0]  # unscaled input
y = res[3]
clf = RandomForestClassifier(n_estimators =80, random_state = 5862)
title_label = '10-fold crossvalidation random forest (80 trees)'
feature_importance(X,y,clf,10,title_label)
plt.suptitle('Target: MCI-MCI: 74, MCI-NL: 74')

In [ ]:
cv_models(g31,drop_lst,target,k)  # no good model

In [ ]:
cv_models(g32,drop_lst,target,k)  # no good model

In [ ]:
cv_models(g33,drop_lst,target,k)

In [ ]:
# 90trees. average weighted f1-score of 10-cross validation:0.813
res = usampling_scale_data(g33,drop_lst,target)     
X = res[0]  # unscaled input
y = res[3]
clf = RandomForestClassifier(n_estimators =90, random_state = 5862)
title_label = '10-fold crossvalidation random forest (90 trees)'
feature_importance(X,y,clf,10,title_label)
plt.suptitle('Target: MCI-AD: 215, MCI-MCI: 215')

In [ ]:
cv_models(g34,drop_lst,target,k)

In [ ]:
# - 90trees. average weighted f1-score of 10-cross validation:0.715
res = usampling_scale_data(g34,drop_lst,target)     
X = res[0]  # unscaled input
y = res[3]
clf = RandomForestClassifier(n_estimators =90, random_state = 5862)
title_label = '10-fold crossvalidation random forest (90 trees)'
feature_importance(X,y,clf,10,title_label)
plt.suptitle('Target: MCI-AD: 72, MCI-NL: 72')

In [ ]:
cv_models(g35,drop_lst,target,k)  # no good model